<a href="https://colab.research.google.com/github/shannonserrao/Arya.ai/blob/main/Vidhya_Census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install deepchecks --User
# !pip install deepchecks --upgrade
# !pip install colorama
# pip install matplotlib==3.1.3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import shap



import warnings
warnings.filterwarnings('ignore')

def custom_formatwarning(msg, *args, **kwargs):
    # ignore everything except the message
    return str(msg) + '\n'
warnings.formatwarning = custom_formatwarning

In [ ]:
from deepchecks.tabular.checks import CalibrationScore
from deepchecks.tabular.checks import ConfusionMatrixReport
from deepchecks.tabular.checks import ModelInfo
# from deepchecks.tabular.checks import PerformanceReport
from deepchecks.tabular.checks import RegressionErrorDistribution
from deepchecks.tabular.checks import RegressionSystematicError
from deepchecks.tabular.checks import RocReport
from deepchecks.tabular.checks import SegmentPerformance
from deepchecks.tabular.checks import SimpleModelComparison
from deepchecks.tabular.checks import MultiModelPerformanceReport
from deepchecks.tabular import Dataset

bold= '\033[1m'
import colorama
from colorama import Fore

# import some classification models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier


# import needed functions
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

## DATA Reading

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Aditya-Mankar/Census-Income-Prediction/master/adult.csv")
df.head(2)

## Data Pre-processing

In [ ]:
questioned_columns = ['workclass' ,'occupation' , 'native.country']

for column in questioned_columns:
        df[column] = df[column].replace({'?': 'Unknown'})

# df.isin(['?']).sum()
df1 = df.copy()
# Scalling
from sklearn.preprocessing import MinMaxScaler
numerical = ['age', 'capital.gain', 'capital.loss', 'hours.per.week', 'fnlwgt']

scaler = MinMaxScaler()
df1[numerical] = scaler.fit_transform(df1[numerical])

# Encoding
df1['sex'] = df1.sex.replace({"Female": 0, "Male": 1})
df1['income'] = df1.income.replace({"<=50K": 0, ">50K": 1})

# Create dummy variables
df1 = pd.get_dummies(df1)

encoded = list(df1.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

In [ ]:
# Partioning the data
X = df1.drop('income', axis=1)
y = df1['income']

# Splitting to training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# categorical_list

In [ ]:
models = {}

# models with default parameter
models['LogisticRegression'] = LogisticRegression()
models['RandomForest'] = RandomForestClassifier()
models['AdaBoost'] = AdaBoostClassifier()

In [ ]:
# Cross validation
for model_name in models:
    model = models[model_name]
    print(model)
    results = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'f1'], return_train_score=True)
    
    print(model_name + ":")
    print("Accuracy:" , 'train: ', results['train_accuracy'].mean(), '| test: ', results['test_accuracy'].mean())
    print("F1-score:" , 'train: ', results['train_f1'].mean(), '| test: ', results['test_f1'].mean())
    print("---------------------------------------------------------")

In [ ]:
# from sklearn.calibration import CalibratedClassifierCV, calibration_curve
# from sklearn.metrics import accuracy_score, brier_score_loss, log_loss

In [ ]:
# # Create classifiers
# lr = LogisticRegression()
# # knn = KNeighborsClassifier()
# # dtc = DecisionTreeClassifier(random_state=42)
# rfc = RandomForestClassifier(random_state=42, verbose=0)

# '''Plot calibration plots'''

# plt.figure(figsize=(15, 10))
# ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
# ax2 = plt.subplot2grid((3, 1), (2, 0))

# ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

# for clf, name in [(lr, 'Logistic'),(rfc, 'RandomForest')]:
    
#     clf.fit(X_train,y_train)
#     prob_pos = clf.predict_proba(X_test)[:, 1]
#     fraction_of_positives, mean_predicted_value = calibration_curve(y_test, prob_pos, n_bins=10)

#     ax1.plot(mean_predicted_value, fraction_of_positives, "s-", label="%s" % (name, ))
#     ax2.hist(prob_pos, range=(0, 1), bins=10, label=name, histtype="step", lw=2)
    

# ax1.set_ylabel("Fraction of positives")
# ax1.set_ylim([-0.05, 1.05])
# ax1.legend(loc="lower right")
# ax1.set_title('Classifier Calibration plots  (reliability curve)', fontsize=20)

# ax2.set_xlabel("Mean predicted value")
# ax2.set_ylabel("Count")
# ax2.legend(loc="upper center", ncol=2)

# plt.tight_layout()
# plt.show()

In [ ]:
# # Classifier
# rfc = RandomForestClassifier(random_state=42, verbose=0)

# # Train
# rfc.fit(X_train,y_train)

# #Calc Probability
# rfc_probs = rfc.predict_proba(X_test)
# rfc_score = log_loss(y_test,rfc_probs)

# # Probability Calibration (sigmoid method)
# sig_rfc = CalibratedClassifierCV(rfc, method="sigmoid", cv="prefit")
# sig_rfc.fit(X_train, y_train)
# sig_rfc_probs = sig_rfc.predict_proba(X_test)
# sig_rfc_score = log_loss(y_test, sig_rfc_probs)

# # Probability Calibration (sigmoid method)
# iso_rfc = CalibratedClassifierCV(rfc, method="isotonic", cv="prefit")
# iso_rfc.fit(X_train, y_train)
# iso_rfc_probs = iso_rfc.predict_proba(X_test)
# iso_rfc_score = log_loss(y_test, iso_rfc_probs)


# print("RFC Log Loss (no calibration) : %1.3f" % rfc_score)
# print("RFC Log Loss (sigmoid) : %1.3f" % sig_rfc_score)
# print("RFC Log Loss (isotonic) : %1.3f" % iso_rfc_score)

In [ ]:
models['RandomForest'].fit(X_train,y_train)

In [ ]:
categorical_list = X.select_dtypes(exclude = ['float','int']).columns.to_list()
ds = Dataset(pd.concat([X_test, y_test], axis=1),cat_features=categorical_list,
            features=X.columns,
            label='income')

%matplotlib inline

def calibration_score(testset, model):
    """
    Function:
    Calculate the calibration curve with brier score for each class
    
    Argument:
    testset: deepcheck wrapped test data
    model: trained model
    """
    try:
        check = CalibrationScore()
        result = check.run(testset, model)
        return result.display[-2]
    except:
        print(bold,Fore.RED +"function calibration score failed")

calibration_score(ds,models['RandomForest'])

- ft == Predicted probablistic score 
- ot == What is your actual prediction(test data point)
- N == No. of Datapoints which needs to be predicted[future prediction instances]
- BS == Lower the score (Near to "0") will be considered best Brier score , given the model.

In [ ]:
train_frac = 0.33
test_frac = 0.33

train_X = df1[[c for c in df1.columns if c != 'income']].iloc[:int(len(df1) * train_frac)].values
train_y = df1.income.iloc[:int(len(df1) * train_frac)].values

test_X = df1[[c for c in df1.columns if c != 'income']].iloc[int(len(df1) * train_frac):int(len(df1) * (train_frac+test_frac))].values
test_y = df1.income.iloc[int(len(df1) * train_frac):int(len(df1) * (train_frac+test_frac))].values

valid_X = df1[[c for c in df1.columns if c != 'income']].iloc[int(len(df1) * (train_frac+test_frac)):].values
valid_y = df1.income.iloc[int(len(df1) * (train_frac+test_frac)):].values

In [ ]:
print("Train_dataset---->",train_X.shape)
print("Train_dataset---->",test_X.shape)
print("Train_dataset---->",valid_X.shape)

In [ ]:
model_to_clf = {
    'rf': RandomForestClassifier,
    'svc': SVC,
    'nb': GaussianNB
    # 'adb': AdaBoostClassifier
               }

In [ ]:
model_to_probs = {}
model_str_to_trained_model = {}

for model_str, model in model_to_clf.items():
    print(model)
    if model == SVC:
        clf = model(probability=True)
    elif model == LogisticRegression:
        clf = model(solver='liblinear')
    else:
        clf = model()
        
    clf.fit(train_X, train_y)
    
    pred_probs_train = clf.predict_proba(train_X)[:,1]
    pred_probs_test = clf.predict_proba(test_X)[:,1]
    pred_probs_valid = clf.predict_proba(valid_X)[:,1]
    
    model_to_probs[model_str] = {'train': pred_probs_train, 'test': pred_probs_test, 'valid': pred_probs_valid}
    
    plt.figure(figsize=(10,4))
    
    plt.subplot(1,2,1)
    sns.distplot(pred_probs_train)
    plt.title(f"{model_str} - train", fontsize=20)
    
    plt.subplot(1,2,2)
    sns.distplot(pred_probs_test)
    plt.title(f"{model_str} - test", fontsize=20)
    
    model_str_to_trained_model[model_str] = clf

In [ ]:
for model_str, pred_prob_dict in model_to_probs.items():
    pred_probs = pred_prob_dict['test']

    pred_probs_space = np.linspace(pred_probs.min(), pred_probs.max(), 10)

    empirical_probs = []
    pred_probs_midpoints = []

    for i in range(len(pred_probs_space)-1):
        empirical_probs.append(np.mean(test_y[(pred_probs > pred_probs_space[i]) & (pred_probs < pred_probs_space[i+1])]))
        pred_probs_midpoints.append((pred_probs_space[i] + pred_probs_space[i+1])/2)

    plt.figure(figsize=(10,4))
    plt.plot(pred_probs_midpoints, empirical_probs, linewidth=2, marker='o')
    plt.title(f"{model_str}", fontsize=20)
    plt.xlabel('predicted prob', fontsize=14)
    plt.ylabel('Fraction of positives', fontsize=14)
    
    plt.plot([0,1],[0,1],linestyle='--',color='gray')
    
    plt.legend(['original', 'ideal'], fontsize=20)

In [ ]:
model_str_to_calibrator = {}

for model_str, pred_prob_dict in model_to_probs.items():
    #train calibration model
    lr_model = LogisticRegression()
    lr_model.fit(pred_prob_dict['test'].reshape(-1,1), test_y)
    
    pred_probs = pred_prob_dict['valid']

    pred_probs_space = np.linspace(pred_probs.min(), pred_probs.max(), 10)

    empirical_probs = []
    pred_probs_midpoints = []

    for i in range(len(pred_probs_space)-1):
        empirical_probs.append(np.mean(valid_y[(pred_probs > pred_probs_space[i]) & (pred_probs < pred_probs_space[i+1])]))
        pred_probs_midpoints.append((pred_probs_space[i] + pred_probs_space[i+1])/2)

    calibrated_probs = lr_model.predict_proba(np.array([0.0]+pred_probs_midpoints+[1.0]).reshape(-1,1))[:,1]
    
    plt.figure(figsize=(10,4))
    plt.plot(pred_probs_midpoints, empirical_probs, linewidth=2, marker='o')
    plt.title(f"{model_str}", fontsize=20)
    plt.xlabel('predicted prob', fontsize=14)
    plt.ylabel('Fraction of positives', fontsize=14)
    
    plt.plot([0.0]+pred_probs_midpoints+[1.0], calibrated_probs, linewidth=2, marker='o')
    
    plt.plot([0,1],[0,1],linestyle='--',color='gray')
    
    plt.legend(['original', 'calibrated', 'ideal'], fontsize=20)
    
    model_str_to_calibrator[model_str] = lr_model

In [ ]:
clf = model_str_to_trained_model['rf']

In [ ]:
lr = model_str_to_calibrator['rf']

In [ ]:
import random
random_value = []
for i in range(107):
    random_value.append(random.random())

In [ ]:
new_sample = np.array([random_value])

In [ ]:
uncal_prob = clf.predict_proba(new_sample)[:,1][0]
print('Uncalibrated Prob:', uncal_prob)

In [ ]:
cal_prob = lr.predict_proba(np.array([[uncal_prob]]))[:,1][0]
print('Calibrated Prob:', cal_prob)

In [ ]:
#train and test frames
#train and test frames
df_class = df1.copy()
target_class = 'income'
y= df_class[target_class]
X= df_class[df_class.columns.difference([target_class])]



##Train Test split
train_df_class, test_df_class = train_test_split(df_class, test_size=0.33, random_state=42)
print(train_df_class.shape)
print(test_df_class.shape)


trainset_class = Dataset(train_df_class, label=target_class, cat_features=categorical_list)
testset_class = Dataset(test_df_class, label=target_class, cat_features=categorical_list)#HERE TARGET IS CATEGORICAL VARIABLE "SPECIES"



## Model fit
clf = RandomForestClassifier(random_state=0, n_estimators=100)
model_class = clf.fit(trainset_class.data[trainset_class.features], trainset_class.data[trainset_class.label_name])

In [ ]:
def model_inference_time(testset,model):
    '''
    Function:
    Check for the inference time of the model
    
    Parameter:
    dataset :Deepchecks wrapped dataframe
    '''
    try:
        check = ModelInferenceTime()
        result = check.run(testset, model)
        return result.display[-1]
    except Exception as e:
        print(bold,Fore.RED +"Model inference failed") 

In [ ]:
model_inference_time(testset_class,model_class)

In [ ]:
from deepchecks.tabular.checks import ModelInferenceTime

In [ ]:
check = ModelInferenceTime(n_samples = 60)
result = check.run(testset_class,model_class)
result.show()

In [ ]:
from deepchecks.checks import TrainTestLabelDrift

In [ ]:
def train_test_label_drift(trainset, testset,max_num_categories):
    """
    Calculate label drift between train dataset and test dataset
    
    Parameter:
    trainset: train data set
    testset: test data set 
    max_num_categories:int: Max number of allowed categories
    """
    try:
        check = TrainTestLabelDrift(max_num_categories)
        result = check.run(trainset, testset)
        return result.display[-1]
    except:
        print("function train test label drift failed")

In [ ]:
train_test_label_drift(trainset_class, testset_class,max_num_categories = None)